In [82]:
import os

import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras import layers
from tensorflow.keras.models import Model

main_folder = ("models/resnet_model")
os.makedirs(f"{main_folder}/layers/keras/", exist_ok=True)
os.makedirs(f"{main_folder}/layers/tflite/", exist_ok=True)

In [83]:
def to_tflite(keras_model, save, save_dir, name):
    # Convert the Keras model to TensorFlow Lite format
    converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
    tflite_model = converter.convert()
    if save:
        with open(f"{save_dir}/{name}.tflite", 'wb') as f:
            f.write(tflite_model)
    return tflite_model

def save_keras(name, model, dir_path):
    model.save(f'{dir_path}/{name}.keras')

def load_keras(name, dir_path):
    return tf.keras.models.load_model(f'{dir_path}/{name}.keras')

def create_keras_submodels(save_dir, model):
    submodels = {}
    
    # Start with the input tensor
    input_tensor = Input(shape=model.input_shape[1:])
    x = input_tensor

    # Iterate through layers and create submodels
    for i, layer in enumerate(model.layers):
        # Pass the input tensor through each layer sequentially
        x = layer(x)
        
        # Create the submodel from the input tensor to the current layer's output
        submodel = Model(inputs=input_tensor, outputs=x)
        submodels[layer.name] = submodel
        
        # Save each submodel to a file
        submodel.save(f'{save_dir}/submodel_{i}.keras')
        print(f"Created Submodel for layer: {layer.name}")
    
    return submodels

    

    
def residual_block(x, filters, kernel_size=3, stride=1):
    # Shortcut
    shortcut = x
    
    # First convolution
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # Second convolution
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    
    # Add shortcut to the block output
    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)
    
    return x

def build_resnet_from_scratch(img_height=224, img_width=224, num_classes=5):
    #inputs = layers.Input(shape=(img_height, img_width, 3))
    model = tf.keras.models.Sequential()
    
    # Initial Conv Layer
    model.add(layers.Conv2D(64, kernel_size=7, strides=2, padding='same', input_shape=(img_height, img_width, 3)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D(pool_size=3, strides=2, padding='same'))
    
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    return model

In [84]:
# Build the model
model = build_resnet_from_scratch()

# Save the model
save_keras(name="resnet_model", model=model, dir_path=main_folder)

# Load the model
model = load_keras(name="resnet_model", dir_path=main_folder)

# Create and save submodels
submodels = create_keras_submodels(model=model, save_dir=f"{main_folder}/layers/keras")

# Convert and save each submodel to TensorFlow Lite format
layer_index = 0
for model_name, model in submodels.items():
    to_tflite(model, save=True, save_dir=f"{main_folder}/layers/tflite", name=f"submodel_{layer_index}")
    layer_index +=1
    
print("All submodels converted to TensorFlow Lite format")

Created Submodel for layer: conv2d_228
Created Submodel for layer: batch_normalization_201
Created Submodel for layer: re_lu_26
Created Submodel for layer: max_pooling2d_21
Created Submodel for layer: dense_36
INFO:tensorflow:Assets written to: C:\Users\fabio\AppData\Local\Temp\tmpq2s5xeq3\assets


INFO:tensorflow:Assets written to: C:\Users\fabio\AppData\Local\Temp\tmpq2s5xeq3\assets


Saved artifact at 'C:\Users\fabio\AppData\Local\Temp\tmpq2s5xeq3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_3997')
Output Type:
  TensorSpec(shape=(None, 112, 112, 64), dtype=tf.float32, name=None)
Captures:
  1778377583184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377583568: TensorSpec(shape=(), dtype=tf.resource, name=None)
INFO:tensorflow:Assets written to: C:\Users\fabio\AppData\Local\Temp\tmpn660lp7q\assets


INFO:tensorflow:Assets written to: C:\Users\fabio\AppData\Local\Temp\tmpn660lp7q\assets


Saved artifact at 'C:\Users\fabio\AppData\Local\Temp\tmpn660lp7q'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_3997')
Output Type:
  TensorSpec(shape=(None, 112, 112, 64), dtype=tf.float32, name=None)
Captures:
  1778377583184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377583568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377586832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377586448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377586256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377572816: TensorSpec(shape=(), dtype=tf.resource, name=None)
INFO:tensorflow:Assets written to: C:\Users\fabio\AppData\Local\Temp\tmpob6q018t\assets


...

Saved artifact at 'C:\Users\fabio\AppData\Local\Temp\tmpob6q018t'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_3997')
Output Type:
  TensorSpec(shape=(None, 112, 112, 64), dtype=tf.float32, name=None)
Captures:
  1778377583184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377583568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377586832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377586448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377586256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377572816: TensorSpec(shape=(), dtype=tf.resource, name=None)
INFO:tensorflow:Assets written to: C:\Users\fabio\AppData\Local\Temp\tmp8v61lrb_\assets


INFO:tensorflow:Assets written to: C:\Users\fabio\AppData\Local\Temp\tmp8v61lrb_\assets


Saved artifact at 'C:\Users\fabio\AppData\Local\Temp\tmp8v61lrb_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_3997')
Output Type:
  TensorSpec(shape=(None, 56, 56, 64), dtype=tf.float32, name=None)
Captures:
  1778377583184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377583568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377586832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377586448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377586256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377572816: TensorSpec(shape=(), dtype=tf.resource, name=None)
INFO:tensorflow:Assets written to: C:\Users\fabio\AppData\Local\Temp\tmpt7jf87rl\assets


INFO:tensorflow:Assets written to: C:\Users\fabio\AppData\Local\Temp\tmpt7jf87rl\assets


Saved artifact at 'C:\Users\fabio\AppData\Local\Temp\tmpt7jf87rl'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_3997')
Output Type:
  TensorSpec(shape=(None, 56, 56, 5), dtype=tf.float32, name=None)
Captures:
  1778377583184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377583568: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377586832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377586448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377586256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377572816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377583760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1778377588368: TensorSpec(shape=(), dtype=tf.resource, name=None)
All submodels converted to TensorFlow Lite format


In [85]:
model.summary()

Model: "functional_170"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_57 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_228 (Conv2D)             │ (None, 112, 112, 64)   │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_201         │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_26 (ReLU)                 │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 56, 56, 5)      │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,053 (39.27 KB)

 Trainable params: 9,925 (38.77 KB)

 Non-trainable params: 128 (512.00 B)